Based in algorithm available at:https://www.pyimagesearch.com/2017/03/20/imagenet-vggnet-resnet-inception-xception-keras/

An interesting implementation is also seen at https://gist.github.com/joelouismarino/a2ede9ab3928f999575423b9887abd14

# Importing libraries

In [1]:
# import the necessary packages
from keras.applications import ResNet50
from keras.applications import InceptionV3
from keras.applications import Xception # TensorFlow ONLY
from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import imagenet_utils
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import numpy as np
import argparse
import cv2

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# construct the argument parse and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-i", "--image", required=True,
#    help="path to the input image")
#ap.add_argument("-model", "--model", type=str, default="vgg16",
#    help="name of pre-trained network to use")
#args = vars(ap.parse_args())
args = {"model":"inception",
        "image0":"images/soccer_ball.jpg",
        "image1":"images/flower01.jpg",
        "image2":"images/flower02.jpg",}

In [3]:
# define a dictionary that maps model names to their classes
# inside Keras
MODELS = {
    "vgg16": VGG16,
    "vgg19": VGG19,
    "inception": InceptionV3,
    "xception": Xception, # TensorFlow ONLY
    "resnet": ResNet50
}
 
# esnure a valid model name was supplied via command line argument
if args["model"] not in MODELS.keys():
    raise AssertionError("The --model command line argument should "
        "be a key in the `MODELS` dictionary")

In [4]:
# initialize the input image shape (224x224 pixels) along with
# the pre-processing function (this might need to be changed
# based on which model we use to classify our image)
inputShape = (224, 224)
preprocess = imagenet_utils.preprocess_input
 
# if we are using the InceptionV3 or Xception networks, then we
# need to set the input shape to (299x299) [rather than (224x224)]
# and use a different image processing function
if args["model"] in ("inception", "xception"):
    inputShape = (299, 299)
    preprocess = preprocess_input

In [5]:
# load our the network weights from disk (NOTE: if this is the
# first time you are running this script for a given network, the
# weights will need to be downloaded first -- depending on which
# network you are using, the weights can be 90-575MB, so be
# patient; the weights will be cached and subsequent runs of this
# script will be *much* faster)
print("[INFO] loading {}...".format(args["model"]))
Network = MODELS[args["model"]]
model = Network(weights="imagenet")

[INFO] loading inception...


# Classifying flower images with the traditional InceptionV3

Creating functions to classify the images and analyse the 5 top categories

In [35]:
def classify_image(image_path,input_shape):
    # load the input image using the Keras helper utility while ensuring
    # the image is resized to `inputShape`, the required input dimensions
    # for the ImageNet pre-trained network
    #print("[INFO] loading and pre-processing image...")
    image = load_img(image_path, target_size=input_shape)
    image = img_to_array(image)
    
    # our input image is now represented as a NumPy array of shape
    # (inputShape[0], inputShape[1], 3) however we need to expand the
    # dimension by making the shape (1, inputShape[0], inputShape[1], 3)
    # so we can pass it through thenetwork
    image = np.expand_dims(image, axis=0)
    
    # pre-process the image using the appropriate function based on the
    # model that has been loaded (i.e., mean subtraction, scaling, etc.)
    image = preprocess(image)
    
    # classify the image
    #print("[INFO] classifying image with '{}'...".format(args["model"]))
    preds = model.predict(image)
    #P = imagenet_utils.decode_predictions(preds)
    
    return preds

def get_files_names(mypath):
    from os import listdir
    from os.path import isfile, join
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    return onlyfiles

In [48]:
images_path = '/home/vfinotti/Pictures/output/short/'
images_list = get_files_names(images_path)


categories_count = np.zeros([1,1000])

for idx, image in enumerate(images_list):
    if (idx%10 == 0):
        print(idx)
    preds = classify_image(images_path+image, inputShape)
    #max_value = max(preds)
    #max_index = preds.index(max_value)
    max_index = np.argmax(preds,axis=1)
    
    categories_count[0, max_index] = categories_count[0, max_index] + 1 

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230


In [53]:
P = imagenet_utils.decode_predictions(categories_count, top=15)
for (i, (imagenetID, label, prob)) in enumerate(P[0]):
    print("{}. {}: {} times".format(i + 1, label, prob))

1. pot: 36.0 times
2. daisy: 33.0 times
3. ant: 17.0 times
4. bee: 14.0 times
5. yellow_lady's_slipper: 13.0 times
6. picket_fence: 11.0 times
7. hip: 9.0 times
8. earthstar: 8.0 times
9. cabbage_butterfly: 8.0 times
10. buckeye: 7.0 times
11. zucchini: 6.0 times
12. hen-of-the-woods: 6.0 times
13. mushroom: 6.0 times
14. rapeseed: 5.0 times
15. strawberry: 4.0 times


In [56]:
images_path = '/home/vfinotti/Pictures/output/full/'
images_list = get_files_names(images_path)


categories_count = np.zeros([1,1000])

for idx, image in enumerate(images_list):
    if (idx%100 == 0):
        print(idx)
    preds = classify_image(images_path+image, inputShape)
    #max_value = max(preds)
    #max_index = preds.index(max_value)
    max_index = np.argmax(preds,axis=1)
    
    categories_count[0, max_index] = categories_count[0, max_index] + 1 

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000


In [57]:
P = imagenet_utils.decode_predictions(categories_count, top=15)
for (i, (imagenetID, label, prob)) in enumerate(P[0]):
    print("{}. {}: {} times".format(i + 1, label, prob))

1. pot: 856.0 times
2. daisy: 427.0 times
3. bee: 282.0 times
4. picket_fence: 208.0 times
5. yellow_lady's_slipper: 196.0 times
6. ant: 170.0 times
7. buckeye: 144.0 times
8. hip: 138.0 times
9. earthstar: 116.0 times
10. hen-of-the-woods: 111.0 times
11. rapeseed: 94.0 times
12. cabbage_butterfly: 89.0 times
13. pinwheel: 85.0 times
14. zucchini: 81.0 times
15. mushroom: 69.0 times


In [7]:
# load the input image using the Keras helper utility while ensuring
# the image is resized to `inputShape`, the required input dimensions
# for the ImageNet pre-trained network
print("[INFO] loading and pre-processing image...")
image = load_img(args["image0"], target_size=inputShape)
image = img_to_array(image)
 
# our input image is now represented as a NumPy array of shape
# (inputShape[0], inputShape[1], 3) however we need to expand the
# dimension by making the shape (1, inputShape[0], inputShape[1], 3)
# so we can pass it through thenetwork
image = np.expand_dims(image, axis=0)
 
# pre-process the image using the appropriate function based on the
# model that has been loaded (i.e., mean subtraction, scaling, etc.)
image = preprocess(image)


# classify the image
print("[INFO] classifying image with '{}'...".format(args["model"]))
preds = model.predict(image)
P = imagenet_utils.decode_predictions(preds)
 
# loop over the predictions and display the rank-5 predictions +
# probabilities to our terminal
for (i, (imagenetID, label, prob)) in enumerate(P[0]):
    print("{}. {}: {:.2f}%".format(i + 1, label, prob * 100))

[INFO] loading and pre-processing image...
[INFO] classifying image with 'inception'...
1. soccer_ball: 100.00%
2. volleyball: 0.00%
3. silky_terrier: 0.00%
4. sea_urchin: 0.00%
5. screw: 0.00%


In [16]:
print(P)

[[('n04254680', 'soccer_ball', 0.9999795), ('n04540053', 'volleyball', 1.2911725e-06), ('n02097658', 'silky_terrier', 9.2055751e-07), ('n02319095', 'sea_urchin', 5.1510028e-07), ('n04153751', 'screw', 4.2401675e-07)]]


In [31]:
# load the input image using the Keras helper utility while ensuring
# the image is resized to `inputShape`, the required input dimensions
# for the ImageNet pre-trained network
print("[INFO] loading and pre-processing image...")
image = load_img(args["image1"], target_size=inputShape)
image = img_to_array(image)
 
# our input image is now represented as a NumPy array of shape
# (inputShape[0], inputShape[1], 3) however we need to expand the
# dimension by making the shape (1, inputShape[0], inputShape[1], 3)
# so we can pass it through thenetwork
image = np.expand_dims(image, axis=0)
 
# pre-process the image using the appropriate function based on the
# model that has been loaded (i.e., mean subtraction, scaling, etc.)
image = preprocess(image)


# classify the image
print("[INFO] classifying image with '{}'...".format(args["model"]))
preds = model.predict(image)
P = imagenet_utils.decode_predictions(preds)
 
# loop over the predictions and display the rank-5 predictions +
# probabilities to our terminal
for (i, (imagenetID, label, prob)) in enumerate(P[0]):
    print("{}. {}: {:.2f}%".format(i + 1, label, prob * 100))

[INFO] loading and pre-processing image...
[INFO] classifying image with 'inception'...
1. picket_fence: 66.42%
2. pot: 4.95%
3. bee: 3.19%
4. cabbage_butterfly: 2.64%
5. vase: 2.03%


In [17]:
# load the input image using the Keras helper utility while ensuring
# the image is resized to `inputShape`, the required input dimensions
# for the ImageNet pre-trained network
print("[INFO] loading and pre-processing image...")
image = load_img(args["image2"], target_size=inputShape)
image = img_to_array(image)
 
# our input image is now represented as a NumPy array of shape
# (inputShape[0], inputShape[1], 3) however we need to expand the
# dimension by making the shape (1, inputShape[0], inputShape[1], 3)
# so we can pass it through thenetwork
image = np.expand_dims(image, axis=0)
 
# pre-process the image using the appropriate function based on the
# model that has been loaded (i.e., mean subtraction, scaling, etc.)
image = preprocess(image)


# classify the image
print("[INFO] classifying image with '{}'...".format(args["model"]))
preds = model.predict(image)
P = imagenet_utils.decode_predictions(preds)
 
# loop over the predictions and display the rank-5 predictions +
# probabilities to our terminal
for (i, (imagenetID, label, prob)) in enumerate(P[0]):
    print("{}. {}: {:.2f}%".format(i + 1, label, prob * 100))

[INFO] loading and pre-processing image...
[INFO] classifying image with 'inception'...
1. cabbage_butterfly: 34.30%
2. bee: 8.07%
3. monarch: 6.79%
4. cardoon: 5.13%
5. pot: 4.40%


In [ ]:
# load the image via OpenCV, draw the top prediction on the image,
# and display the image to our screen
#orig = cv2.imread(args["image"])
#(imagenetID, label, prob) = P[0][0]
#cv2.putText(orig, "Label: {}, {:.2f}%".format(label, prob * 100),
#    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
#cv2.imshow("Classification", orig)
#cv2.waitKey(0)

# Classifying flower images with the finetuned InceptionV3

## Building the network

In [12]:
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense

# build the inception network
base_model = InceptionV3(weights=None, include_top=False,
                         input_shape = (150, 150, 3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

#top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
#model.add(top_model)
model = Model(input= base_model.input, output= top_model(base_model.output))

model.load_weights('./weights/inceptionv3_finetuned.h5')

Model loaded.


/usr/lib/python3/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`


# Classifying the images

In [ ]:
images_path = '/home/vfinotti/Pictures/output/full/'
images_list = get_files_names(images_path)


categories_count = np.zeros([1,1000])

for idx, image in enumerate(images_list):
#    if (idx%100 == 0):
#        print(idx)
    preds = classify_image(images_path+image, inputShape)
    #max_value = max(preds)
    #max_index = preds.index(max_value)
    max_index = np.argmax(preds,axis=1)
    categories_count[0, max_index] = categories_count[0, max_index] + 1 
    
#P = imagenet_utils.decode_predictions(categories_count, top=15)
#for (i, (imagenetID, label, prob)) in enumerate(P[0]):
#    print("{}. {}: {} times".format(i + 1, label, prob))